In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. 加载数据
data = pd.read_csv('./data/total_load_actual.csv')

# 2. 检查数据是否包含 NaN
if data.isnull().any().any():
    print("Data contains NaN values. Filling missing values...")
    data = data.ffill()  # 使用前值填充缺失数据

# 3. 将数据列转换为字典格式
data_dict = {}
for col in data.columns:
    data_dict[col] = data[col].tolist()

total_acual_load_list = data_dict['total load actual']

# 4. 归一化数据
scaler = MinMaxScaler(feature_range=(0, 1))
total_acual_load_list_scaled = scaler.fit_transform(np.array(total_acual_load_list).reshape(-1, 1)).flatten()

# 5. 设定序列长度和样本数
seq_length = 24
num_samples = len(total_acual_load_list_scaled) - seq_length

# 6. 初始化输入和输出数据
x_processed = []
y_processed = []

window_size = 8 # 滑动窗口大小
i = 0 
while i * window_size + seq_length - 1 < len(total_acual_load_list_scaled):
    x_processed.append(total_acual_load_list_scaled[i * window_size:i * window_size + seq_length - 2])
    y_processed.append(total_acual_load_list_scaled[i * window_size + seq_length - 1])
    i += 1

# 7. 转换为NumPy数组
x_processed = np.array(x_processed)
y_processed = np.array(y_processed)

# 8. 转换为Tensor
x_processed_tensor = torch.tensor(x_processed, dtype=torch.float32)
y_processed_tensor = torch.tensor(y_processed, dtype=torch.float32).reshape(-1, 1)

# 9. 数据形状检查
x_processed_tensor = x_processed_tensor.unsqueeze(-1)  # 将输入扩展为三维张量 (样本数量, seq_length-1, input_dim)

# 10. 数据划分: 80% 训练集,20% 测试集
train_size = 0.8
x_train, x_test, y_train, y_test = train_test_split(x_processed_tensor, y_processed_tensor, test_size=1-train_size, random_state=42)

# 11. 定义模型
class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(CNN_LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # CNN部分
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        # LSTM部分
        self.lstm = nn.LSTM(input_size=64, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)

        # 全连接层
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # 调整维度顺序
        x = self.cnn(x)  # CNN特征提取
        x = x.permute(0, 2, 1)  # 调整回LSTM输入格式
        x = x.repeat(1, 24, 1)  # Repeat Vector
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # 只取最后一个时间步
        return out

# 12. 定义模型
input_dim = 1
hidden_dim = 32
output_dim = 1
num_layers = 1
model = CNN_LSTM(input_dim, hidden_dim, output_dim, num_layers)
learning_rate = 0.002

# 13. 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# 14. 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 15. 加载权重文件(如果存在)
weight_file = './weights/cnn_lstm_model.pth'
hyperparams_file = './weights/hyperparams.txt'

# 如果有预训练的权重文件,则加载
if os.path.exists(weight_file):
    print(f"Loading pre-trained model from {weight_file}")
    model.load_state_dict(torch.load(weight_file))
else:
    print("No pre-trained model found, starting from scratch.")

# 16. 训练模型
num_epochs = 1000
train_loss = []
print('Training model...')
start_time = time.time()

for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch", mininterval=30, maxinterval=30):
    model.train()
    optimizer.zero_grad()

    # 移动数据到GPU
    x_train_device = x_train.to(device)
    y_train_device = y_train.to(device)

    output = model(x_train_device)
    loss = criterion(output, y_train_device)
    loss.backward()
    optimizer.step()

    train_loss.append(loss.item())
    if (epoch+1) % (num_epochs // 10) == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
training_time = end_time - start_time
print(f'Training completed in {training_time:.2f} seconds')

# 17. 保存模型和超参数
os.makedirs('./weights', exist_ok=True)
torch.save(model.state_dict(), weight_file)

# 保存超参数
hyperparams = {
    'input_dim': input_dim,
    'hidden_dim': hidden_dim,
    'output_dim': output_dim,
    'num_layers': num_layers,
    'num_epochs': num_epochs,
    'learning_rate': learning_rate
}

with open(hyperparams_file, 'w') as f:
    for key, value in hyperparams.items():
        f.write(f"{key}: {value}/n")

print(f"Model and hyperparameters saved to {weight_file} and {hyperparams_file}")

Data contains NaN values. Filling missing values...
Using device: cpu
Loading pre-trained model from ./weights/cnn_lstm_model.pth
Training model...


Training:   7%|▋         | 67/1000 [00:30<07:02,  2.21epoch/s]

Epoch [100/1000], Loss: 0.0072


Training:  20%|█▉        | 198/1000 [01:39<06:56,  1.93epoch/s]

Epoch [200/1000], Loss: 0.0029


...

Epoch [800/1000], Loss: 0.0020


Training:  88%|████████▊ | 884/1000 [08:20<01:12,  1.61epoch/s]

Epoch [900/1000], Loss: 0.0019


Training: 100%|██████████| 1000/1000 [09:33<00:00,  1.74epoch/s]

Epoch [1000/1000], Loss: 0.0015
Training completed in 573.32 seconds
Model and hyperparameters saved to ./weights/cnn_lstm_model.pth and ./weights/hyperparams.txt


In [2]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 计算各类指标的函数
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = np.mean(np.abs(y_true - y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    rmse = np.sqrt(mse)
    r2 = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))
    smape = np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
    
    return mse, mae, mape, rmse, r2, smape

# 15. 测试模型
model.eval()
with torch.no_grad():
    predicted = model(x_test).cpu().numpy()

# 16. 反归一化预测结果
predicted_original_scale = scaler.inverse_transform(predicted)
y_test_original_scale = scaler.inverse_transform(y_test.cpu().numpy())

# 17. 计算归一化前的指标
mse_original, mae_original, mape_original, rmse_original, r2_original, smape_original = calculate_metrics(y_test_original_scale, predicted_original_scale)

# 18. 计算归一化后的指标
y_test_np = y_test.cpu().numpy().flatten()
predicted_np = predicted.flatten()
mse_normalized, mae_normalized, mape_normalized, rmse_normalized, r2_normalized, smape_normalized = calculate_metrics(y_test_np, predicted_np)

# 19. 打印结果
print('+=============+=============+=============+=============+')
print('|   Metrics (Original Scale)   |   Value   |   Metrics (Normalized Scale)   |   Value   |')
print('+=============+=============+=============+=============+')
metrics = [
    ('MSE', mse_original, mse_normalized),
    ('MAE', mae_original, mae_normalized),
    ('MAPE', mape_original, mape_normalized),
    ('RMSE', rmse_original, rmse_normalized),
    ('R²', r2_original, r2_normalized),
    ('SMAPE', smape_original, smape_normalized)
]

for metric, original_value, normalized_value in metrics:
    print(f'| {metric:<15} | {original_value:>26.6f} | {metric} (Normalized){"":<2} | {normalized_value:>26.6f} |')

print('+=============+=============+=============+=============+')


+=============+=============+=============+=============+
|   Metrics (Original Scale)   |   Value   |   Metrics (Normalized Scale)   |   Value   |
+=============+=============+=============+=============+
| MSE             |              933893.125000 | MSE (Normalized)   |                   0.001769 |
| MAE             |                 639.548096 | MAE (Normalized)   |                   0.027838 |
| MAPE            |                   2.322192 | MAPE (Normalized)   |                   7.828744 |
| RMSE            |                 966.381470 | RMSE (Normalized)   |                   0.042064 |
| R²              |                   0.932650 | R² (Normalized)   |                   0.932650 |
| SMAPE           |                   0.023168 | SMAPE (Normalized)   |                   0.074990 |
+=============+=============+=============+=============+


In [4]:
import plotly.graph_objs as go
import plotly.offline as pyo

# 1. 训练损失的交互式可视化
train_loss_fig = go.Figure()

# 添加训练损失曲线
train_loss_fig.add_trace(go.Scatter(
    x=list(range(1, num_epochs + 1)),  # 横坐标是训练的轮数
    y=train_loss,  # 纵坐标是训练过程中的损失
    mode='lines',  # 线形显示
    name='Training Loss',  # 曲线的标签
    line=dict(color='royalblue', width=3, dash='dash')  # 设置颜色、宽度和虚线样式
))

# 更新图表的布局
train_loss_fig.update_layout(
    title='CNN-LSTM:Training Loss Curve',  # 图表标题
    title_font=dict(size=20, family='Times New Roman', color='black'),  # 设置标题字体
    xaxis=dict(title='Epoch', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # X轴标签
    yaxis=dict(title='Loss', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # Y轴标签
    plot_bgcolor='whitesmoke',  # 设置图表背景色
    paper_bgcolor='whitesmoke',  # 设置图表外部背景色
    font=dict(family='Times New Roman', size=12, color='black'),  # 字体设置
    legend=dict(
        x=0.5, y=1.05,  # 设置图例位置
        xanchor='center',  # 图例水平居中
        yanchor='bottom',  # 图例垂直居上
        traceorder='normal',  # 图例顺序
        orientation='h',  # 图例显示为水平排列(即一行多列)
        font=dict(family='Times New Roman', size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.5)',  # 图例背景颜色
        bordercolor='black', borderwidth=1  # 图例边框颜色和宽度
    ),
    showlegend=True,  # 显示图例
    width=1200,  # 设置图表宽度
    height=600  # 设置图表高度
)

# 设置图表边缘的阴影效果
train_loss_fig.update_layout(
    xaxis=dict(showgrid=True, zeroline=False, showline=True, linecolor='gray'),  # X轴样式
    yaxis=dict(showgrid=True, zeroline=False, showline=True, linecolor='gray'),  # Y轴样式
    margin=dict(l=50, r=50, t=50, b=50)  # 设置图表的边距
)

train_loss_fig.show()

# 2. 预测与实际结果的交互式可视化
# 创建实际结果的曲线
timestep = 100
actual_trace = go.Scatter(
    x=list(range(timestep)),  # 横坐标是时间步
    y=y_test_original_scale[:timestep].flatten(),  # 纵坐标是实际测试数据(前100个数据点)
    mode='lines+markers',  # 线条和数据点显示
    name='Actual',  # 曲线名称
    line=dict(color='mediumseagreen', width=3, shape='spline'),  # 设置颜色、宽度和曲线类型
    marker=dict(symbol='circle', size=8, color='white', line=dict(color='mediumseagreen', width=2))  # 数据点样式(空心)
)

# 创建预测结果的曲线
predicted_trace = go.Scatter(
    x=list(range(timestep)),  # 横坐标是时间步
    y=predicted_original_scale[:timestep].flatten(),  # 纵坐标是模型的预测数据(前100个数据点)
    mode='lines+markers',  # 线条和数据点显示
    name='Predicted',  # 曲线名称
    line=dict(color='indianred', width=3, dash='dot'),  # 设置颜色、宽度和虚线样式
    marker=dict(symbol='x', size=8, color='white', line=dict(color='indianred', width=2))  # 数据点样式(空心)
)

# 创建包含实际和预测数据的图表
comparison_fig = go.Figure(data=[actual_trace, predicted_trace])

# 更新布局设置
comparison_fig.update_layout(
    title='CNN-LSTM:Actual vs Predicted',  # 图表标题
    title_font=dict(size=20, family='Times New Roman', color='black'),  # 标题字体
    xaxis=dict(title='Time Step', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # X轴标签
    yaxis=dict(title='Load', title_font=dict(size=14, family='Times New Roman', color='black'), gridcolor='lightgray'),  # Y轴标签
    plot_bgcolor='whitesmoke',  # 图表背景颜色
    paper_bgcolor='whitesmoke',  # 外部背景颜色
    font=dict(family='Times New Roman', size=12, color='black'),  # 字体设置
    legend=dict(
        x=0.5, y=1.05,  # 设置图例位置
        xanchor='center',  # 图例水平居中
        yanchor='bottom',  # 图例垂直居上
        traceorder='normal',  # 图例顺序
        orientation='h',  # 图例显示为水平排列(即一行多列)
        font=dict(family='Times New Roman', size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.5)',  # 图例背景颜色
        bordercolor='black', borderwidth=1  # 图例边框颜色和宽度
    ),
    showlegend=True,  # 显示图例
    width=1200,  # 设置图表宽度
    height=600  # 设置图表高度
)

# 为图表添加渐变背景色
comparison_fig.update_layout(
    paper_bgcolor='rgba(240, 240, 240, 0.9)',  # 外部背景渐变色
    plot_bgcolor='rgba(255, 255, 255, 0.9)',  # 图表区域背景渐变色
)

# 显示预测与实际结果的图表
comparison_fig.show()

# 将图片保存到 ./results/ 目录下
pyo.plot(train_loss_fig, filename='./results/train_loss_curve.html', auto_open=False)
pyo.plot(comparison_fig, filename='./results/actual_vs_predicted.html', auto_open=False)

'./results/actual_vs_predicted.html'